In [136]:
import os
import numpy as np
import pandas as pd

from scipy.interpolate import LinearNDInterpolator, CloughTocher2DInterpolator,griddata,RBFInterpolator

from mycolorpy import colorlist as mcp
import matplotlib.pyplot as plt

import plotly
import plotly.graph_objects as go


from mylibs import tools, functions as f
from mylibs.tools import Experiment

pd.set_option('mode.chained_assignment', None)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('font', size=10)
plt.rc('axes', titlesize=16)
plt.rc('axes', labelsize=15)
plt.rc('legend', fontsize=10)
plt.rcParams["figure.figsize"] = (10, 7)

In [127]:
R

0.06666666666666667

In [124]:
D_mean=15000

In [5]:
%matplotlib qt

# Collect

In [6]:
def to_df(experiment:Experiment):
    df= experiment.d
    k = 1
    df['Viscosity'] = k * (df['x'] + 273.15) / (df['y'] * 0.001)
    df['compound'] = experiment.info['compound']
    df['rho'] = experiment.info['rho']
    df['w'] = experiment.info['w']
    df['D0'] = experiment.info['D0']
    df['E'] = experiment.info['E']
    df.rename(columns={'x': 'Temperature', 'y': 'D'}, inplace=True)
    return df
    


In [7]:
# OCM
ocm_list = []
for i in range(9):
    path = f'D:\WORKS\Diplom-work\Experiments\OCM_viscosity\OCM{i}'
    file = [f for f in os.listdir(path) if '.hdf5' in f][0]

    exp = Experiment()
    exp.load_hdf5(f'{path}\{file}')
    ocm_list.append(to_df(exp))

ocm = pd.concat(ocm_list)

In [129]:
ocm_list[0]

,time,D,Viscosity_verbose,Temperature,Temperature_verbose,Viscosity,compound,rho,w,D0,E
0,1.0,831.980284,OK,13.8,OK,344.9,ocm,1.72,0.0,5.805356e+13,59372.5935
1,2.0,831.980284,OK,13.8,OK,344.9,ocm,1.72,0.0,5.805356e+13,59372.5935
2,3.0,831.980284,OK,13.8,OK,344.9,ocm,1.72,0.0,5.805356e+13,59372.5935
3,4.0,830.775912,image_sweep_check,13.8,OK,345.4,ocm,1.72,0.0,5.805356e+13,59372.5935
4,5.0,830.775912,image_sweep_check,13.8,OK,345.4,ocm,1.72,0.0,5.805356e+13,59372.5935
...,...,...,...,...,...,...,...,...,...,...,...
18502,19333.0,6863.238512,OK,40.5,OK,45.7,ocm,1.72,0.0,5.805356e+13,59372.5935
18503,19334.0,6818.478261,OK,40.5,OK,46.0,ocm,1.72,0.0,5.805356e+13,59372.5935
18504,19335.0,6878.289474,OK,40.5,image_sweep_check,45.6,ocm,1.72,0.0,5.805356e+13,59372.5935
18505,19336.0,6863.238512,OK,40.5,OK,45.7,ocm,1.72,0.0,5.805356e+13,59372.5935


In [8]:
# PETA
peta_list = []
for i in range(5):
    path = f'D:\WORKS\Diplom-work\Experiments\PETA,TMAG_viscosity\PETA{i}'
    file = [f for f in os.listdir(path) if '.hdf5' in f][0]

    exp = Experiment()
    exp.load_hdf5(f'{path}\{file}')
    peta_list.append(to_df(exp))
    
peta = pd.concat(peta_list)

In [9]:
# Butanol
but= Experiment()
but.load_hdf5('D:\Works\Diplom-work\Experiments\BUTANOL\data_BUT_up.hdf5')
but= to_df(but)


In [10]:
all_df = pd.concat([ocm, peta,but]).reset_index(drop=True)[[
    'compound', 'w', 'Temperature', 'time', 'D', 'Viscosity', 'rho', 'D0', 'E', 'Viscosity_verbose'
]]


In [109]:
df = (all_df[['compound','w','Temperature','D','Viscosity','rho','D0','E']]
    .groupby(by=['compound','w','Temperature'])
    .mean()
    .reset_index()
)

# 2D

In [10]:
# 2D surface
compound_filter = (df['compound'] == 'ocm')
# |(df['compound'] == 'butanol')
data = df[compound_filter]

xy= data[['Temperature','w']]
z= data['D']
X = np.linspace(min(xy['Temperature']),max(xy['Temperature']))
Y = np.linspace(min(xy['w']),max(xy['w']))
X ,Y= np.meshgrid(X, Y)
f = LinearNDInterpolator(xy,z)
Z = f(X,Y)
plt.pcolormesh(X, Y, Z)

In [122]:
# Diffusion plots
name = 'OCM-2'
# +' + Butanol-1'

compound_filter = (
    (df['compound'] == 'peta')
    |(df['compound'] == 'butanol')
) #yapf: disable

# Plot
data = df[compound_filter]
grouped = data.groupby(by=['w'])
colors_list = mcp.gen_color(cmap="turbo", n=len(grouped))

fig, ax = plt.subplots()
for _, group in grouped:
    # print()
    w = group['w'].mean()
    T = group['Temperature']
    color = colors_list.pop()

    D = group['D'] / D_mean
    ax.scatter(
        T,
        D,
        c=color,
        s=2,
    )

    E = group['E'].mean()
    D0 = group['D0'].mean() / D_mean
    D_ols = D0 * np.exp(-E / (8.314 * (T+273.15)))
    ax.plot(
        T,
        D_ols,
        label=f"{w: 5.2f}",
        c=color,
    )

    ax.set_title(f"{name}: diffusion plots")
    ax.set_xlabel('Temperature [C]')
    ax.set_ylabel('Diffusion')
plt.legend()

# 3D T,w=D,rho

In [120]:
fig = go.Figure()
fig.update_layout(
    # hovermode="x unified",
    margin=dict(l=0, r=0, b=0, t=0, pad=0),
    scene=dict(
        xaxis=dict(title='Temperature [C]'),
        yaxis=dict(title='W mass [%]'),
        zaxis=dict(title='Diffusion'), 
    ))
D_mean = df['D'].mean()
quality=21

# OCM
compound_filter = (df['compound'] == 'ocm') 
# |(df['compound'] == 'butanol')

data = df[compound_filter]
xy = data[['Temperature', 'w']]
x = data['Temperature']
y = data['w']
z = data['D'] / D_mean


f = LinearNDInterpolator(xy, z)

x1 = np.linspace(x.min(), x.max(), quality)
y1 = np.linspace(y.min(), y.max(), quality)
X, Y = np.meshgrid(x1, y1)
R = np.ones_like(X)
Z = f(X,Y)

fig.add_trace(go.Surface(
    name='OCM-2',
    x=x1,
    y=y1,
    z=Z,
    colorscale='Reds',
    surfacecolor=R,
    showscale=False,
    showlegend=True,
))

# PETA
compound_filter = (df['compound'] == 'peta') 
# |(df['compound'] == 'butanol')
data = df[compound_filter]

xy = data[['Temperature', 'w']]
x = data['Temperature']
y = data['w']
z =  data['D'] / D_mean

Rbf
fz = CloughTocher2DInterpolator(xy, z)

x1 = np.linspace(x.min(), x.max(), quality)
y1 = np.linspace(y.min(), y.max(), quality)
X, Y = np.meshgrid(x1, y1)


Z = fz(X, Y)

fig.add_trace(go.Surface(
    name='PETA',
    x=x1,
    y=y1,
    z=Z,
    colorscale='Blues',
    surfacecolor=R,
    showscale=False,
     showlegend=True,
))
fig.show()

# Rework massively

## Correct E

In [ ]:
import os
for i in range(5):

    exp = tools.Experiment()
    path = f'D:\Works\Diplom-work\Experiments\PETA,TMAG_viscosity\PETA{i}'
    file = [file for file in os.listdir(path) if '.hdf5' in file][0]
    exp.load_hdf5(*tools.input_path(f'{path}\{file}'))
    exp.log.pop()
    info, result, func = tools.regress(exp)
    exp.set_info(**info)
    del exp.info['d_D0']
    del exp.info['d_E']
    exp.save_hdf5()

In [ ]:
with pd.HDFStore('D:\Works\Diplom-work\Experiments\OCM_viscosity\OCM0\data_OCM0_up.hdf5') as file:
    log = file.get_storer('data').attrs.log 
    info= file.get_storer('data').attrs.info 

In [ ]:
info

In [ ]:
exp2 =tools.Experiment()
exp2.load_hdf5(*tools.input_path())
exp2.log

In [ ]:
exp2.info

## Plots

In [146]:
import os
for i in range(9):

    exp = tools.Experiment()
    path = f'D:\Works\Diplom-work\Experiments\OCM_viscosity\OCM{i}'
    file = [file for file in os.listdir(path) if '.hdf5' in file][0]
    exp.load_hdf5(tools.input_path(f'{path}\{file}'))
    exp.log.pop()

    info, result, func = tools.regress(exp)
    print(exp.info)
    x = np.linspace(13, 42, 100) + 273.15
    ols_res = tools.Experiment(
        pd.DataFrame({
            'x': x, 'y': func(x), 'time': x * 0
        }),
        'interpolated',
    )

    ols_res.apply(f.K_to_C)

    os.makedirs(f'{path}\Plots', exist_ok=True)
    tools.comparation_plot(
        exp,
        ols_res,
        title='OLS_Diffusion',
        xlabel='Temperature [C]',
        ylabel='D [m2/s]',
        # interactive=True,
        save=True,
    )


{'compound': 'ocm', 'rho': 1.72, 'w': 0.0, 'D0': 58053559827657.89, 'f_statistic': 3029495.381638645, 'r2': 0.9951642727866552, 'E': 59372.593499571696, 'dD0': 2256959362103.371, 'dE': 95.76658562539887}
{'compound': 'ocm', 'rho': 1.58, 'w': 2.39, 'D0': 4791470034128.69, 'f_statistic': 956673.2052230232, 'r2': 0.99644926238456, 'E': 52367.62704465405, 'dD0': 285210909989.9419, 'dE': 150.38745445442328}
{'compound': 'ocm', 'rho': 1.57, 'w': 4.98, 'D0': 5275514951246.829, 'f_statistic': 1144382.9737901262, 'r2': 0.994941753543445, 'E': 52025.030106846985, 'dD0': 291007531032.3843, 'dE': 136.5651908117361}
{'compound': 'ocm', 'rho': 1.55, 'w': 7.6, 'D0': 1235647924812.8499, 'f_statistic': 780086.0321886932, 'r2': 0.9973139416615985, 'E': 47746.14564175396, 'dD0': 75254119756.09753, 'dE': 151.90559726174615}
{'compound': 'ocm', 'rho': 1.54, 'w': 10.07, 'D0': 262135768384.52338, 'f_statistic': 970656.2292308109, 'r2': 0.9966670165622645, 'E': 43340.69261907823, 'dD0': 12851204078.0027, 'dE'